# Configuração de Constantes

In [2]:
# Quantas voltas serão utilizadas para prever a próxima volta
TRAILING_LAPS = 5
# Tamanho do conjunto de teste
TEST_SIZE = 200

# Selecionando dados

Iremos utilizar todos os tempos de volta de um determinado GP (Bahrein) de um ano (2021) para prever os tempos de volta desse mesmo GP no ano seguinte (2022)

In [3]:
# Conexão com o banco de dados
import mariadb
import configparser

cfg_parser = configparser.ConfigParser()
with open('database.config', 'r') as config_file:
    cfg_parser.read_file(config_file)
    host = cfg_parser.get('database', 'host')
    port = cfg_parser.get('database', 'port')
    user = cfg_parser.get('database', 'user')
    password = cfg_parser.get('database', 'password')
    name = cfg_parser.get('database', 'name')


conn = mariadb.connect(
    host=host,
    port=int(port),
    user=user,
    password=password,
    database=name
)

In [4]:
cursor = conn.cursor()
# Bahrein foi a primeira etapa da temporada 2021
cursor.execute("SELECT id FROM races WHERE year = 2021 AND round = 1")
race_id = cursor.fetchone()[0]

In [16]:
import numpy as np
# Retornando o id de todos os pilotos que registraram uma volta nessa corrida
cursor.execute("SELECT DISTINCT driver_id FROM lap_times WHERE race_id = ?", [race_id])
drivers_id = np.asarray(cursor.fetchall()).flatten()
laps_by_each_driver = dict()
for id in drivers_id:
    cursor.execute("SELECT milliseconds FROM lap_times WHERE race_id = ? AND driver_id = ?", [race_id, id])
    laps_by_each_driver[id] = np.asarray(cursor.fetchall()).flatten()

In [58]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(np.asarray([0, 180000]).reshape(-1, 1))
inputs = []
outputs = []
for driver_index in laps_by_each_driver.keys():
    driver_laps = laps_by_each_driver[driver_index]
    for i in range(len(driver_laps) - TRAILING_LAPS):
        inputs.append(scaler.transform(driver_laps[i:i+TRAILING_LAPS].reshape(-1, 1)))
        outputs.append(scaler.transform(driver_laps[i+TRAILING_LAPS].reshape(-1, 1)))

In [59]:
inputs = np.asarray(inputs)
outputs = np.asarray(outputs)

# Criando rede LSTM

## Definindo a estrutura

A rede LSTM precisa ter um input=TRAILING_LAPS e um output=1

In [62]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

# Modelo LSTM
model = Sequential()
model.add(LSTM(128, input_shape=(TRAILING_LAPS, 1), return_sequences=True))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(1))

2023-06-13 01:04:12.330796: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-13 01:04:12.331813: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-13 01:04:12.332579: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

## Treinando rede neural

In [63]:
# Compilando o modelo
model.compile(loss='mse', optimizer='adam')
model.fit(inputs, outputs, epochs=1000)

Epoch 1/1000


2023-06-13 01:04:15.339678: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-13 01:04:15.340835: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-13 01:04:15.341688: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

30/30 [==============================] - 2s 5ms/step - loss: 0.0494
Epoch 2/1000
30/30 [==============================] - 0s 5ms/step - loss: 0.0030
Epoch 3/1000
30/30 [==============================] - 0s 5ms/step - loss: 0.0020
Epoch 4/1000
30/30 [==============================] - 0s 5ms/step - loss: 0.0019
Epoch 5/1000
30/30 [==============================] - 0s 6ms/step - loss: 0.0019
Epoch 6/1000
30/30 [==============================] - 0s 5ms/step - loss: 0.0017
Epoch 7/1000
30/30 [==============================] - 0s 5ms/step - loss: 0.0019
Epoch 8/1000
30/30 [==============================] - 0s 5ms/step - loss: 0.0019
Epoch 9/1000
30/30 [==============================] - 0s 5ms/step - loss: 0.0017
Epoch 10/1000
30/30 [==============================] - 0s 6ms/step - loss: 0.0016
Epoch 11/1000
30/30 [==============================] - 0s 5ms/step - loss: 0.0016
Epoch 12/1000
30/30 [==============================] - 0s 5ms/step - loss: 0.0014
Epoch 13/1000
30/30 [=================

In [71]:
for offset in range(0, 20):
    input_test = laps_by_each_driver[1][offset : offset + 5]
    output_test = laps_by_each_driver[1][offset + 5]
    input_test = scaler.transform(input_test.reshape(-1, 1))
    output_test = scaler.transform(output_test.reshape(-1, 1))
    input_test = input_test.reshape(1, TRAILING_LAPS, 1)
    print(f"Predicted: {scaler.inverse_transform(model.predict(input_test))}, Real: {scaler.inverse_transform(output_test)}")
    print(f"Difference: {scaler.inverse_transform(model.predict(input_test)) - scaler.inverse_transform(output_test)}")

1/1 [==============================] - 0s 16ms/step
Predicted: [[99153.12]], Real: [[96169.]]
1/1 [==============================] - 0s 13ms/step
Difference: [[2984.1171875]]
1/1 [==============================] - 0s 14ms/step
Predicted: [[96613.805]], Real: [[96123.]]
1/1 [==============================] - 0s 13ms/step
Difference: [[490.8046875]]
1/1 [==============================] - 0s 13ms/step
Predicted: [[95751.98]], Real: [[96075.]]
1/1 [==============================] - 0s 14ms/step
Difference: [[-323.0234375]]
1/1 [==============================] - 0s 16ms/step
Predicted: [[94902.96]], Real: [[96148.]]
1/1 [==============================] - 0s 14ms/step
Difference: [[-1245.0390625]]
1/1 [==============================] - 0s 14ms/step
Predicted: [[95221.875]], Real: [[96082.]]
1/1 [==============================] - 0s 14ms/step
Difference: [[-860.125]]
1/1 [==============================] - 0s 14ms/step
Predicted: [[96042.17]], Real: [[96143.]]
1/1 [============================